In [1]:
from urllib.request import Request, urlopen, HTTPError
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import pandas as pd
import csv

keyword = ['데이터 사이언티스트', '데이터애널리스트', '데이터 엔지니어']
page = 1
front_url = 'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=recently&searchword='
mid_url = '&recruitPage='
back_url = '&recruitSort=relation&recruitPageCount=100&inner_com_type=&company_cd=0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C9%2C10&show_applied=&quick_apply=&except_read=&ai_head_hunting='
url_data = []
company_name = []
position_name = []
post_title = []
company_position = []

driver = webdriver.Chrome("chromedriver")

# url들을 찾는 크롤링코드
for search_keyword in keyword:
    driver.get(front_url+search_keyword+mid_url+str(page)+back_url)
    ct_result = driver.find_element(By.XPATH, '//*[@id="recruit_info"]/div[1]/span').text
    total_result = re.sub(r'[^0-9]', '', ct_result)
    page_ct = int(int(total_result)/100)

    for pages in range(1, page_ct+2):
        try:
            driver.get(front_url+search_keyword+mid_url+str(pages)+back_url)
            driver.implicitly_wait(time_to_wait=60)

            div_1 = driver.find_elements(By.CLASS_NAME, "job_tit")
            for item in div_1:
                tmp_url = item.find_element(By.TAG_NAME, "a").get_attribute('href')
                url_data.append(tmp_url)
                tmp_title = item.find_element(By.TAG_NAME, "a").text
                post_title.append(tmp_title)

            div_2 = driver.find_elements(By.CLASS_NAME, "corp_name")
            for item in div_2:
                tmp_cn = item.find_element(By.TAG_NAME, "a").text
                company_name.append(tmp_cn)
                position_name.append(search_keyword)

            time.sleep(3)

        except Exception as e:
            pass

driver.close()
print('검색결과 수 : ', len(url_data))

검색결과 수 :  5638


In [2]:
from urllib.request import Request, urlopen, HTTPError
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
import pandas as pd
import csv

ct = 1      # 진행상황 파악 위해서 count
e_ct = 1    # error갯수 파악 위한 count
content = []

driver = webdriver.Chrome("chromedriver")

for url in url_data:
    driver.get(url)
    driver.implicitly_wait(time_to_wait=60)

    try:
        address = driver.find_element(By.CLASS_NAME, 'address').text
        company_position.append(address)

    except:
        company_position.append('None')
        pass

    try:
        
        driver.switch_to.frame("iframe_content_0")
        contents = driver.find_element(By.CLASS_NAME, "user_content").text
        content.append(contents)
        
        print(ct)
        ct += 1
        time.sleep(3)

    except:
        print(e_ct)
        e_ct += 1
        content.append('None')
        continue

driver.close()
saramin_df = pd.DataFrame({'회사명' : company_name, '직무' : position_name, '링크' : url_data, '공고내용' : content, '회사주소' : company_position})
saramin_df.to_csv("saramin_df.csv", index = False)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
